# Image Multithresholding
#### A Python 3 library to find thresholds in the gray levels' histogram of an image, which may be used in segmentation.

##### Installing the package
Run the cell bellow to install the `image-multithresholding` package in your system.

In [ ]:
#!python3 -m pip install image-multithresholding
!python3 -m pip install --upgrade --index-url https://test.pypi.org/simple --no-deps image-multi-thresholding-test01==0.0.6

### Histogram of gray levels
For a grayscale image in the luminance range 1 (black) to $L$ (white), $H(i)$ denotes the number of pixels in the image with gray level $i \in [1, \dots,L]$. 
The normalized gray-level histogram is defined by
$$p_i=\frac{H(i)}{N},$$
where $N$ is the total number of pixels.

##### Importing the modules
It is necessary to import the following modules in order to run the examples presented in this tutorial.

In [ ]:
# Run this cell to import the necessary packages to run the example.
#!python3 -m pip install matplotlib numpy
from IPython.display import Image
from matplotlib import pyplot as plt
import numpy as np
from image_multi_thresholding.base import load_image

##### Loading the image to analyze
Load the image of interest using the `base.load_image` function.
In what follows, the analysis is based on the reference image named **exImage**.

In [ ]:
image = load_image('SARsynthetic.jpg')

# Preview
Image('SARsynthetic.jpg', width=200, height=200)

##### Example of histogram
The code in the following cell produces and visualizes the histogram of the grays levels of the reference image. 

In [ ]:
from image_multi_thresholding.base import _image_histogram

histogram = _image_histogram(image)

# Preview
%matplotlib inline
x = np.arange(0, len(histogram))
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
ax.plot(x, histogram)
ax.set_title('Histogram')
ax.set_xlabel('Gray level')
ax.set_ylabel('Pixel count')

##### Example of probability density function (*pdf*)
The code in the following cell produces and visualizes the *pdf* of the grays levels of the reference image. 

In [ ]:
from image_multi_thresholding.base import _image_probabilities

prob = _image_probabilities(image)

# Preview
%matplotlib inline
x = np.arange(0, len(prob))
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
ax.plot(x, prob)
ax.set_title('Probabilities')
ax.set_xlabel('Gray level')
ax.set_ylabel('%')

### Classes defined by a set of thresholds 
Given a set of gray levels $t_1, \dots ,t_{k-1}$ and considering $t_0=0$ and $t_k=L$, the collection $[1, \dots , L]$ can be partitioned into $k$ classes 
$$C_j=\{t_{j-1}+1, \dots ,t_j\},$$
for $j=1, \dots, k$.

### Basic statistical concepts
Consider the class $C_j$, for $j \in \{1, \dots, k\}$. 
The following is defined:
<ul>
<li> The probability of occurrence $$w_j=\sum_{i \in C_j} p_i.$$ </li>
<li> The mean of levels $$\mu_j=\sum_{i \in C_j} \frac{ip_i}{w_j}.$$ </li>
<li> The variance of levels $$\sigma_j^2=\sum_{i \in C_j} \frac{(i-\mu_j)^2p_i}{w_j}.$$ </li>
<li> The $n$th central moment of levels $$(\mu_n)_j=\sum_{i \in C_j} \frac{(i-\mu_j)^np_i}{w_j}.$$
Notice that $(\mu_2)_j=\sigma_j^2$. </li>
<li> The skewness of levels $(\beta_1)_j=\frac{(\mu_3)_j}{(\mu_2)_j^{3/2}}.$$ </li>
</ul>